# **Importing the dependencies**


In [28]:
!pip install transformers
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import sklearn

# **IMPORTING THE DATASET**

In [3]:
df = pd.read_csv('/content/IMDB Dataset.csv', error_bad_lines=False, engine="python")

<ipython-input-3-220bc3de695c>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/IMDB Dataset.csv', error_bad_lines=False, engine="python")
Skipping line 10275: unexpected end of data


# **DATA VISUALIZATION AND PREPROCESSING**

In [4]:
df.describe()

,review,sentiment
count,10273,10273
unique,10255,2
top,I've been strangely attracted to this film sin...,positive
freq,2,5153


In [5]:
df.shape

(10273, 2)

In [6]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [7]:
df['sentiment'].value_counts()

positive    5153
negative    5120
Name: sentiment, dtype: int64

In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
df.duplicated().sum()

18

In [10]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

# **Lowercasing**

In [11]:
df['review'] = df['review'].str.lower()

# **Removing HTML tags**

In [12]:
import re
def tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

df['review'] = df['review'].apply(tags)
df['review'][1]

'a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well done.'

# **Removing Punctuations**

In [13]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
exclude = string.punctuation
def punct(text):
    return text.translate(str.maketrans('','',exclude))

df['review'] = df['review'].apply(punct)
df['review'][1]

'a wonderful little production the filming technique is very unassuming very oldtimebbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great masters of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwells murals decorating every surface are terribly well done'

# **Spelling Correction**

In [15]:
from textblob import TextBlob
def spell(text):
    blob = TextBlob(text)
    return blob.correct().string

# **Removing StopWords**

In [16]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('english')
df['review'] = df['review'].apply(lambda x:[word for word in x.split() if word not in sw]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Handeling Emojis**

In [17]:
pip install emoji

In [18]:
import emoji
def demoji(text):
    review = emoji.demojize(text)
    return review

df['review'] = df['review'].apply(demoji)
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive
...,...,...
10268,rest us love good sentimental emotional story ...,positive
10269,michael rooker decent actor business lead exce...,negative
10270,glories snare unlikely outing hanzo would able...,positive
10271,one films whatever reason clicked everything r...,positive


# **Label Encoding of Target Column**

In [19]:
X = df.iloc[:,0:1]
y = df['sentiment']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

# **Train-Test Split**

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [21]:
X_train.shape, X_test.shape

((8204, 1), (2051, 1))

# **Using Bag Of Words**

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

X_train_bow.shape, X_test_bow.shape

((8204, 73644), (2051, 73644))

# ***Naive Bayes***

In [23]:
# Importing the Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit the Naive Bayes model
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_bow, y_train)

# Predictions
y_pred_nb = naive_bayes.predict(X_test_bow)

# Evaluate Naive Bayes
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_score_nb = f1_score(y_test, y_pred_nb)

print("Naive Bayes Accuracy:", accuracy_nb)
print("Naive Bayes Precision:", precision_nb)
print("Naive Bayes Recall:", recall_nb)
print("Naive Bayes F1 Score:", f1_score_nb)


Naive Bayes Accuracy: 0.8347147732813262
Naive Bayes Precision: 0.8659685863874346
Naive Bayes Recall: 0.7967244701348748
Naive Bayes F1 Score: 0.8299046663321626


# ***RNN***

In [24]:
# RNN
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train['review'])
X_train_seq = tokenizer.texts_to_sequences(X_train['review'])
X_test_seq = tokenizer.texts_to_sequences(X_test['review'])

# Pad sequences to have the same length
X_train_seq = pad_sequences(X_train_seq, maxlen=200)
X_test_seq = pad_sequences(X_test_seq, maxlen=200)

# Create an RNN model
model = keras.Sequential([
    keras.layers.Embedding(10000, 64, input_length=200),
    keras.layers.LSTM(64),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the RNN model
model.fit(X_train_seq, y_train, epochs=5, batch_size=64)

# Evaluate RNN
loss, accuracy_rnn = model.evaluate(X_test_seq, y_test)
y_pred_rnn = (model.predict(X_test_seq) > 0.5).astype(int)
precision_rnn = precision_score(y_test, y_pred_rnn)
recall_rnn = recall_score(y_test, y_pred_rnn)
f1_score_rnn = f1_score(y_test, y_pred_rnn)

print("RNN Accuracy:", accuracy_rnn)
print("RNN Precision:", precision_rnn)
print("RNN Recall:", recall_rnn)
print("RNN F1 Score:", f1_score_rnn)


Epoch 1/5
129/129 [==============================] - 26s 170ms/step - loss: 0.4980 - accuracy: 0.7554
Epoch 2/5
129/129 [==============================] - 25s 196ms/step - loss: 0.2181 - accuracy: 0.9220
Epoch 3/5
129/129 [==============================] - 22s 168ms/step - loss: 0.1109 - accuracy: 0.9654
Epoch 4/5
129/129 [==============================] - 24s 186ms/step - loss: 0.0651 - accuracy: 0.9804
Epoch 5/5
65/65 [==============================] - 2s 28ms/step
RNN Accuracy: 0.8395904302597046
RNN Precision: 0.8202348690153568
RNN Recall: 0.8747591522157996
RNN F1 Score: 0.8466200466200466


In [30]:
# Replace 'new_text' with the actual text you want to analyze
new_text = "This is a sample text for sentiment analysis."

# Transform the new text using the same CountVectorizer
new_text_bow = cv.transform([new_text]).toarray()

# Predict sentiment using the Naive Bayes model
predicted_sentiment_nb = naive_bayes.predict(new_text_bow)

# Analyze the prediction
if predicted_sentiment_nb == 1:
    print("Naive Bayes predicts positive sentiment for the new text.")
else:
    print("Naive Bayes predicts negative sentiment for the new text.")


Naive Bayes predicts negative sentiment for the new text.


In [33]:
# Assuming you have a new text sample in a variable called 'new_text'

# Preprocess the new text (apply the same preprocessing as done for the training data)

# Tokenize and preprocess the new text
new_text_tokens = tokenizer.encode_plus(new_text, add_special_tokens=True, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

# Predict sentiment using the BERT model
with torch.no_grad():
    new_text_input_ids, new_text_attention_mask = new_text_tokens['input_ids'], new_text_tokens['attention_mask']
    new_text_logits = model(new_text_input_ids, attention_mask=new_text_attention_mask).logits
    predicted_sentiment_bert = torch.argmax(new_text_logits, dim=1).item()

# Analyze the prediction
if predicted_sentiment_bert == 1:
    print("BERT predicts positive sentiment for the new text.")
else:
    print("BERT predicts negative sentiment for the new text.")


BERT predicts positive sentiment for the new text.
